# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [61]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [80]:
weather_df = pd.read_csv('../WeatherPy/output_data.csv')
weather_df = pd.DataFrame(weather_df)
weather_df =weather_df[weather_df['Lat'] <= 90 ]
weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ushuaia,40,AR,1580592886,71,-54.80,-68.30,53.60,28.86
1,1,at-bashi,42,KG,1580592889,46,41.17,75.81,-1.52,10.71
2,2,dikson,51,RU,1580592890,97,73.51,80.55,-14.55,12.93
3,3,rikitea,100,PF,1580592891,78,-23.12,-134.97,79.65,12.75
4,4,tazovskiy,65,RU,1580592893,96,67.47,78.70,-18.85,13.69
...,...,...,...,...,...,...,...,...,...,...
455,455,batagay,71,RU,1580593590,95,67.63,134.63,-24.03,1.74
456,456,teguldet,95,RU,1580593591,98,57.31,88.17,-3.28,3.96
457,457,salinopolis,68,BR,1580593592,78,-0.61,-47.36,81.27,15.77
458,458,strizhi,99,RU,1580593593,96,58.46,49.29,15.64,4.76


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [63]:
gmaps.configure(api_key=g_key)

In [67]:
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig = gmaps.figure()
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [83]:
ideal_temp =weather_df.loc[(weather_df['Max Temp'] >= 65) & (weather_df['Max Temp'] <= 80)]
ideal_hum = ideal_temp.loc[(ideal_temp['Humidity'] >= 40) & (ideal_temp['Humidity'] <= 65)]
wind= ideal_hum.loc[(ideal_hum["Wind Speed"]<10)]
hotel_df = wind.loc[(wind['Cloudiness']<= 25)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [111]:
hotel_df["Hotel Name"] = ""
hotel_df

/Users/Julia/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
18,18,clearlake,1,US,1580592711,55,38.96,-122.63,69.01,3.33,
130,130,maneadero,0,MX,1580593096,53,31.72,-116.57,71.01,1.99,
270,270,ouesso,11,CG,1580593307,54,1.61,16.05,77.23,1.97,
284,284,otane,1,NZ,1580593331,58,-39.88,176.63,73.00,5.99,
327,327,taltal,0,CL,1580593391,65,-25.40,-70.48,69.08,7.29,


In [141]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": 'hotel',
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params)
    hotel = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [142]:
hotel_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
18,18,clearlake,1,US,1580592711,55,38.96,-122.63,69.01,3.33,Hotel Healdsburg
130,130,maneadero,0,MX,1580593096,53,31.72,-116.57,71.01,1.99,Hotel Boutique Valentino
270,270,ouesso,11,CG,1580593307,54,1.61,16.05,77.23,1.97,Hotel Mindongo Ouesso
284,284,otane,1,NZ,1580593331,58,-39.88,176.63,73.00,5.99,Cape Kidnappers
327,327,taltal,0,CL,1580593391,65,-25.40,-70.48,69.08,7.29,Hotel Mi Tampi


In [144]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [150]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
hotel_layer = gmaps.marker_layer(
    locations, info_box_content = hotel_info)
fig.add_layer(hotel_layer)


# Display Map
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))